In [1]:
!pip install -q datasets transformers==4.31.0 trl==0.4.7 accelerate==0.21.0 peft==0.4.0 scipy wandb bitsandbytes==0.40.2 sentencepiece


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

import wandb
from getpass import getpass
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from datasets import load_dataset

In [3]:
model_name = 'TinyPixel/Llama-2-7B-bf16-sharded'
dataset_name = 'odunola/complete_foodie_dataset'


In [4]:
dataset = load_dataset(dataset_name)['train'].train_test_split(test_size = 0.2)

In [5]:
train_dataset, eval_dataset = dataset['train'], dataset['test']

In [6]:
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1

lora_config = LoraConfig(
    lora_alpha = lora_alpha,
    lora_dropout = lora_dropout,
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj'],
    r = lora_r,
    bias = 'none',
    task_type = 'CAUSAL_LM'
)

In [7]:
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

bnb_config = BitsAndBytesConfig(
    load_in_4bit = use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=bnb_4bit_compute_dtype,
    bnb_4bit_use_double_quant = use_nested_quant,
)

In [8]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    device_map = 'auto'
)

In [9]:
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast = False)
tokenizer.pad_token = tokenizer.eos_token

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [10]:
output_dir = "./results"
num_train_epochs = -1
fp16 = False
bf16 = False
per_device_train_batch_size = 16
per_device_eval_batch_size = 16
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 3e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "constant"
max_steps = 5000
warmup_ratio = 0.03
group_by_length = True
save_steps = 100
logging_steps = 25
max_seq_length = None
packing = True

In [11]:
arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size= per_device_eval_batch_size,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="all",
    evaluation_strategy="steps",
    eval_steps=200,  # Evaluate every 20 steps
    save_strategy = 'steps',
    save_safetensors = True,
    packing = True
)

In [12]:
trainer = SFTTrainer(
    model = base_model,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    peft_config = lora_config,
    dataset_text_field = 'text',
    tokenizer = tokenizer,
    args = arguments
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/41601 [00:00<?, ? examples/s]

Map:   0%|          | 0/10401 [00:00<?, ? examples/s]

In [13]:
trainer.train()

wandb: Currently logged in as: jenrola2292. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
200,0.771300,1.034294
400,0.759500,1.034886
600,0.775300,1.010839
800,0.752300,1.006678
1000,0.758000,0.997395
1200,0.740800,0.998571
1400,0.741000,0.995829
1600,0.782000,0.991529
1800,0.749600,0.984636
2000,0.757400,0.985098


KeyboardInterrupt: 

In [16]:
train_dataset[]['text']

['Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nDescribe how two different cultures could view the same topic in different ways.\n\n### Response:\nFor example, two different cultures may view the concept of family differently. In one culture, the idea of family may be seen as a fundamental building block of a larger community, while in another culture, the concept of family could be seen as a private and personal unit.',
 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nGenerate an example of a school report card, including grades and comments.\n\n### Input:\nName: John Smith\nGrade: 10th\n\n### Response:\nJohn Smith is completing his 10th grade at Rolling Hills High School. His grades are as follows: \nEnglish: B+\nMath: A-\nHistory: B\nScience: C+\n\nJohn is a hardworking stu

In [47]:
from IPython.display import display, Markdown

def make_inference(question):
  batch = tokenizer(f"Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{question}\n### Response:\n", return_tensors='pt')

  with torch.cuda.amp.autocast():
    output_tokens = trainer.model.generate(**batch, max_new_tokens=200)

  display(Markdown((tokenizer.decode(output_tokens[0], skip_special_tokens=True))))

In [48]:
import torch

In [51]:
make_inference('who is jon bellion')

TypeError: make_inference() got an unexpected keyword argument 'token'

In [54]:
from huggingface_hub import notebook_login

In [57]:
notebook_login()

In [59]:
trainer.push_to_hub('testing done!', token = True)

OSError: Looks like you do not have git-lfs installed, please install. You can install from https://git-lfs.github.com/. Then run `git lfs install` (you only have to do this once).

In [62]:
trainer.model.push_to_hub('odunola/results')

adapter_model.bin:   0%|          | 0.00/170M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/odunola/results/commit/d897f959200470775cc5f186126e0caade6c1d24', commit_message='Upload model', commit_description='', oid='d897f959200470775cc5f186126e0caade6c1d24', pr_url=None, pr_revision=None, pr_num=None)